In [1]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import torch
import sys
import os
import numpy as np
%matplotlib inline

#生成等边三角形
corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
triangle = tri.Triangulation(corners[:, 0], corners[:, 1])
#每条边中点位置
midpoints = [(corners[(i + 1) % 3] + corners[(i + 2) % 3]) / 2.0 for i in range(3)]
def xy2bc(xy, tol=1.e-3):
    #将三角形顶点的笛卡尔坐标映射到重心坐标系
    s = [(corners[i] - midpoints[i]).dot(xy - midpoints[i]) / 0.75 for i in range(3)]
    return np.clip(s, tol, 1.0 - tol)
#有了重心坐标，可以计算Dirichlet概率密度函数的值
class Dirichlet(object):

    def __init__(self, alpha):
        from math import gamma
        from operator import mul
        from functools import reduce
        self._alpha = np.array(alpha)
        self._coef = gamma(np.sum(self._alpha)) / reduce(mul, [gamma(a) for a in self._alpha]) #reduce:sequence连续使用function

    def pdf(self, x):
        #返回概率密度函数值
        from operator import mul
        from functools import reduce
        return self._coef * reduce(mul, [xx ** (aa - 1) for (xx, aa)in zip(x, self._alpha)])
def draw_pdf_contours(dist, nlevels=200, subdiv=8, **kwargs):
    import math

    #细分等边三角形网格
    refiner = tri.UniformTriRefiner(triangle)
    trimesh = refiner.refine_triangulation(subdiv=subdiv)
    pvals = [dist.pdf(xy2bc(xy)) for xy in zip(trimesh.x, trimesh.y)]

    plt.tricontourf(trimesh, pvals, nlevels, **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    # 制作数据集
root = "/data/public/renhaoye/morphics/"
subdirs = [os.path.join(root, d) for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
for subdir in subdirs:
    sys.path.append(subdir)
from dataset.galaxy_dataset import *
from utils import schemas
from utils.utils import *
from models.utils import *
import random
from torch.backends import cudnn
def init_rand_seed(rand_seed):
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(rand_seed)  # 为所有GPU设置随机种子
    np.random.seed(rand_seed)
    random.seed(rand_seed)
    cudnn.benchmark = False
    cudnn.deterministic = True
init_rand_seed(1926)
question_answer_pairs = gz2_pairs
dependencies = gz2_and_decals_dependencies
schema = schemas.Schema(question_answer_pairs, dependencies)

In [23]:
img = torch.Tensor(load_img("/data/public/renhaoye/morphics/dataset/in_decals/agmtn/119.85429851358965_28.803799075693085.fits"))
# img = torch.Tensor(load_img("/data/public/renhaoye/1.fits"))
device = "cuda:1"
model = torch.load("/data/public/renhaoye/morphics/pth/model_7.pt", map_location=device)

In [24]:
predict, stn= model(img.unsqueeze(0).to(device))
predicted = softmax_output(predict,schema.question_index_groups)
fits.writeto(f'/data/public/renhaoye/pth.fits', stn[0].cpu().detach().numpy(), overwrite=True)
predicted

tensor([[6.5469e-02, 9.0383e-01, 3.0702e-02, 1.4356e-02, 9.8564e-01, 9.6272e-01,
         3.7279e-02, 3.5844e-02, 2.5483e-01, 7.0933e-01, 4.6856e-04, 4.2918e-03,
         2.0436e-01, 7.7056e-01, 2.0323e-02, 3.3501e-01, 6.2239e-01, 4.2596e-02,
         6.7756e-02, 2.5931e-01, 6.7294e-01, 5.9973e-01, 3.3460e-01, 6.5678e-02,
         1.3915e-02, 2.9976e-01, 2.5195e-01, 1.2044e-01, 4.1427e-02, 2.7251e-01,
         6.1549e-01, 2.4472e-01, 8.8493e-02, 5.1292e-02]], device='cuda:1',
       grad_fn=<CopySlices>)

In [25]:
def cal_total_data_uc(model, x, num_samples):
    batch_size = 1
    num_classes = 34
    probs = torch.zeros(batch_size, num_classes, num_samples, device=x.device)
    entropy = torch.zeros(batch_size, num_classes, num_samples, device=x.device)
    for i in range(num_samples):
        model.train()
        with torch.no_grad():
            output, _ = model(x)
        prob = softmax_output(output,schema.question_index_groups)
        probs[:, :, i] = prob
        entropy[:, :, i] = -prob * torch.log(prob)
    expected_probs = probs.mean(dim=2)
    variance_probs = ((probs - expected_probs.unsqueeze(2))**2).mean(dim=2)
    data_uc = entropy.mean(dim=2)
    total_uc = -expected_probs * torch.log(expected_probs)
    return total_uc, data_uc, variance_probs
a, b, v = cal_total_data_uc(model, img.unsqueeze(0).to(device), 100)
print(a.sum())
print(b.sum())

tensor(4.6946, device='cuda:1')
tensor(4.5991, device='cuda:1')


In [21]:
a, b = cal_total_data_uc(model, img.unsqueeze(0).to(device), 100)
print(a.sum())
print(b.sum())

tensor(5.7741, device='cuda:1')
tensor(5.6144, device='cuda:1')


In [26]:
a.sum()-b.sum()

tensor(0.0955, device='cuda:1')

In [34]:
a-b

tensor([[2.9112e-03, 1.2356e-02, 6.4185e-04, 4.1880e-03, 1.6154e-04, 3.3116e-02,
         2.3260e-02, 1.6658e-03, 5.8825e-03, 7.0371e-04, 7.6591e-04, 1.9776e-03,
         7.8146e-03, 7.8296e-03, 2.3421e-03, 1.7969e-02, 3.6734e-03, 2.4829e-03,
         7.7283e-04, 3.4017e-03, 2.1174e-04, 4.4586e-03, 9.2777e-03, 6.3401e-03,
         1.1938e-03, 6.5775e-03, 7.6583e-04, 1.6780e-04, 2.7347e-05, 2.5910e-03,
         7.1472e-03, 1.6027e-02, 2.8804e-03, 1.6888e-03]], device='cuda:1')

In [7]:
output_ = []
for mc_run in range(100):
    output, _ = model(img.unsqueeze(0).to(device))
    output_.append(softmax_output(output,schema.question_index_groups))
predict = torch.mean(torch.stack(output_), dim=0)

In [8]:
predict

tensor([[1.8373e-01, 8.1627e-01, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 9.4517e-02, 9.0548e-01, 0.0000e+00, 3.9221e-04, 8.5420e-03,
         2.6249e-01, 7.2858e-01, 0.0000e+00, 4.3372e-01, 5.6628e-01, 0.0000e+00,
         2.4300e-01, 7.5700e-01, 0.0000e+00, 7.3903e-01, 2.6097e-01, 0.0000e+00,
         2.2968e-02, 3.4617e-01, 3.8609e-01, 1.7332e-01, 7.1457e-02, 0.0000e+00,
         7.2108e-01, 2.3048e-01, 4.8438e-02, 0.0000e+00]], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [5]:
# 将img的batchsize变成100以重复100次预测
img = img.unsqueeze(0).repeat(100,1,1,1).to(device)
predict, _ = model(img)
# output_.append(softmax_output(output,schema.question_index_groups))
# predict = torch.mean(torch.stack(output_), dim=0)

In [8]:
predict = softmax_output(predict,schema.question_index_groups)
predict = torch.mean(predict, dim=0)

In [9]:
predict

tensor([1.8776e-01, 8.1224e-01, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 9.6046e-02, 9.0395e-01, 0.0000e+00, 4.0164e-04, 8.7288e-03,
        2.7191e-01, 7.1896e-01, 0.0000e+00, 4.1080e-01, 5.8920e-01, 0.0000e+00,
        2.4650e-01, 7.5350e-01, 0.0000e+00, 7.3532e-01, 2.6468e-01, 0.0000e+00,
        2.3337e-02, 3.5907e-01, 3.9032e-01, 1.6347e-01, 6.3808e-02, 0.0000e+00,
        7.2410e-01, 2.2828e-01, 4.7621e-02, 0.0000e+00], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [10]:
target_layer = list(list(model.children())[0].children())[-2]
target_layer

AdaptiveAvgPool2d(output_size=1)